In [43]:
import random

def read_maps(size, n_sp, n_tar, id):
    map_start = []
    map_end = []
    T = 0
    with open(f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\start\\graph_start_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_{id}.txt", "r") as file_start:
        T = int(file_start.readline())
        for line in file_start.readlines():
            # map(function/type, iterable_object)，把測資用 map 轉整數，然後轉成 list
            map_start += [list(map(int, line.split(',')))]
    
    with open(f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\end\\graph_end_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_{id}.txt", "r") as file_end:
        T = int(file_end.readline())
        for line in file_end.readlines():
            map_end += [list(map(int, line.split(',')))]

    return T, map_start, map_end



def set_parameter(size, n_sp, n_tar, id):
    # size = (3,3)，但 grid_size 因為是 int，所以我先取了第一維長度
    grid_size = size[0]
    num_cargos = size[0] * size[1] - n_sp
    num_robots = num_cargos   # robot 的數量我先跟總貨物設定的一樣，你可以再改
    alpha = 0.001
    return grid_size, num_cargos, num_robots, alpha



def set_node(size, map_start, map_end, num_robots):
    initial_positions = []
    cargo_init_pos = []
    cargo_goal_pos = []
    
    non_target = []
    for i in range(size[0]):
        for j in range(size[1]):
            # print((map_start[i][j]-1, i, j))
            if map_start[i][j] > 0: cargo_init_pos.append((map_start[i][j]-1, i, j))
            elif map_start[i][j] < 0: non_target.append((-1, i, j))
    for non in non_target:
        cargo_init_pos.append((non[0]+1+len(cargo_init_pos), non[1], non[2]))
    cargo_init_pos = sorted(cargo_init_pos, key=lambda x:x[0])

    cargo_goal_pos = [(map_end[i][j]-1, i, j) for i in range(size[0]) for j in range(size[1]) if map_end[i][j] > 0]
    cargo_goal_pos = sorted(cargo_goal_pos, key=lambda x:x[0])
    cargos, targets = (len(cargo_init_pos), len(cargo_goal_pos))
    for _ in range(cargos - targets):
        cargo_goal_pos.append((None, None, None))
    

    # 先暫時將機器人的位置設定為所有貨物底下，一人一台
    initial_positions = [t for t in cargo_init_pos]

    # 如果要改機器人數量設定，可以改用這幾行，隨機產生 num_robots台的機器人位置（但是上一行要 comment 起來）
    # all_pos = [(x, y) for x in range(size[0]) for y in range(size[1])]
    # rand_pos = random.sample(all_pos, num_robots)
    # initial_positions = [(i, x, y) for i, (x, y) in enumerate(rand_pos)]

    return initial_positions, cargo_init_pos, cargo_goal_pos



def preprocess(size, n_sp, n_tar, id):
    T, map_start, map_end = read_maps(size, n_sp, n_tar, id)
    grid_size, num_cargos, num_robots, alpha = set_parameter(size, n_sp, n_tar, id)
    initial_positions, cargo_init_pos, cargo_goal_pos = set_node(size, map_start, map_end, num_robots)

    return T, map_start, map_end, grid_size, num_cargos, num_robots, alpha, initial_positions, cargo_init_pos, cargo_goal_pos

